In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import functools
import operator

import polars as pl

from football.data import read_parquet
from football.optimize import Optimizer, roster_to_team_picks_idx_map
from football.platforms.sleeper import League

# Data

In [ ]:
# Specify points column; this depends on your league
points_col = "pts_half_ppr"

# Get projections
proj = read_parquet("weekly_projections.parquet")
proj = proj[["player_id", "week", points_col]].drop_nulls("pts_half_ppr")

# Get notable players
season_proj = proj.group_by("player_id").agg(season=pl.col(points_col).sum())
player_ids = season_proj.filter(pl.col("season") > 0)[["player_id"]].sort("player_id")

# Pivot
proj = proj.pivot(on="week", index="player_id")
proj = player_ids.join(proj, on="player_id", how="left").fill_null(0)

proj

In [ ]:
# Get players
players = read_parquet("players.parquet")
players = players.join(player_ids, on="player_id", how="inner").sort("player_id")
players = players[["player_id", "full_name", "team", "position", "fantasy_positions"]]

players

In [ ]:
# Join
data = players.join(proj, on="player_id", how="inner", coalesce=True)

# Downselect valid positions
valid_positions = {"QB", "RB", "WR", "TE", "K", "DEF"}
data = data.filter(pl.col("position").is_in(valid_positions))
data = data.with_row_index()

data

# Optimization

In [ ]:
# Get league
league = League("<league_id>")

# Get draft
draft = league.get_draft()
# OR
# from football.platforms.sleeper import Draft
# draft = Draft("<draft_id>")

In [ ]:
# Initialize optimizer
optimizer = Optimizer(
    pos_col="position",
    week_cols=[str(i + 1) for i in range(17)],  # inclusive; skip week 18 as the championship is week 17
    roster_size=league.roster_size,
    # Note: position constraints aren't necessarily the league settings
    min_pos_const={"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DEF": 1},
    max_pos_const={"QB": 2, "RB": 6, "WR": 6, "TE": 2, "K": 1, "DEF": 1},
)

In [ ]:
# Set state manually
# team = 0
# team_picks_idx_map = {k: set() for k in range(draft.league_size)}
# team_picks_idx_map[team] |= {149, 357, 448}

# Set state with live data
draft.update_picks()
team = draft.get_next_pick()
# OR
# team = 3  # for auction; note: one-indexed
rosters = draft.get_rosters()
team_picks_idx_map = roster_to_team_picks_idx_map(data, rosters)

# Get all picks
all_picks_idx = set(functools.reduce(operator.iadd, [list(v) for v in team_picks_idx_map.values()], []))

team

In [ ]:
%%time

# Optimize once
curr_picks_idx = team_picks_idx_map[team]
roster_idx = optimizer.optimal_roster(data, curr_picks_idx, all_picks_idx, verbose=False)

In [ ]:
# View solution
team_data = data[sorted(roster_idx)]
team_data = team_data.with_columns(
    season_proj=pl.sum_horizontal(optimizer.week_cols), new_pick=~pl.col("index").is_in(curr_picks_idx)
)
team_data = team_data.sort(["new_pick", "season_proj"], descending=True)

with pl.Config(tbl_rows=-1):
    display(team_data[["index", "player_id", "full_name", "team", "position", "season_proj", "new_pick"]])
    print("Sum of weekly projections for fantasy season:")
    display(team_data[optimizer.week_cols].sum())